
# Indices of the BSE crop:

***Original shape: $(2048, 3072)$***

    - X (columns)
        - Total crop width: dx = 2220 
        - slice(476, 2696)
    - Y (rows)
        - Total crop height: dy = 1665
        - slice(175, 1840)

Once this crop is complete, a resize transform was applied through ***scikit-image*** to get to a target size of $(483, 644)$

# Indices of the EBSD crop:

***Original shape: $(484,654)$***

    - X (columns)
        - Total crop width: dx = 644
        - slice(10, None)
    - Y (rows)
        - Total crop height: dy = 483
        - slice(None, -1)


___
___

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from skimage import io, transform, exposure
%matplotlib widget

import core

def generate_gif(array):
    fig = plt.figure()
    im = plt.imshow(numpy_3d_array[0, :, :],
                    animated=True,
                    cmap='turbo',
                    vmin=array.min(),
                    vmax=array.max())
    def init():
        im.set_data(array[0, :, :])
        return im,
    def animate(i):
        im.set_array(array[i, :, :])
        return im,
    anim = animation.FuncAnimation(fig,
                                   animate,
                                   init_func=init,
                                   frames=array.shape[0],
                                   interval=200,
                                   blit=True)
    # anim.save("test.gif")
    plt.colorbar(label='turbo', shrink=0.75)
    plt.tight_layout()
    plt.show()

In [ ]:
h5 = h5py.File("D:/Research/CoNi_16/Data/3D/CoNi16_aligned.dream3d", 'r')
data = h5["DataContainers/ImageDataContainer/CellData/SEM"][:,:,:,0]
h5.close()
ebsd = io.imread("Slice420_CoNi16/ebsd.tif")
# EBSD CROP: remove last row (0:-1) and remove first 10 columns (10:)
data = data[:, :-1, 10:]
print(data.shape, ebsd.shape)

In [ ]:
plt.close()
bse_full = io.imread("Slice420_CoNi16/aligned_0039.tif", as_gray=True)
bse_full = np.around(255*bse_full/bse_full.max(), 0).astype(np.uint8)
bse_resized = io.imread("Slice420_CoNi16/bse.tif")

y_selected = 175
x_selected = None
dx = 2224
dy = int(dx*0.75)

if x_selected is None:
    start_x = np.arange(350, bse_full.shape[1] - dx - 350, 1, dtype=int)
    start_y = np.ones(start_x.shape[0], dtype=int)*y_selected
else:
    start_y = np.arange(100, bse_full.shape[0] - dy - 100, 1, dtype=int)
    start_x = np.ones(start_y.shape[0], dtype=int)*x_selected

diffs = []
slices_x = []
slices_y = []
for i in range(start_x.shape[0]):
    slx = slice(start_x[i], start_x[i]+dx)
    sly = slice(start_y[i], start_y[i]+dy)
    bse_full_crop = bse_full[sly, slx]
    bse_adjusted = transform.resize(bse_full_crop, ebsd.shape, anti_aliasing=False)
    # bse_adjusted = bse_adjusted.astype(np.uint8)
    bse_adjusted = np.around(255*bse_adjusted/bse_adjusted.max(), 0).astype(np.uint8)
    true = bse_resized[:200, :200]
    alt = bse_adjusted[:200, :200]
    slices_x.append(slx)
    slices_y.append(sly)
    diffs.append(abs(np.sum(true-alt)))
diffs = np.array(diffs)
slices_x = np.array(slices_x)
slices_y = np.array(slices_y)

In [ ]:
i = diffs.argmin()
print(f"Mininum:\n  x->{slices_x[i]}\n  end->{slices_y[i]}\n  val->{diffs[i]}")
dx = 2220
dy = int(dx*0.75)
sly = slice(175, 175+dy)
slx = slice(476, 476+dx)#slices_x[i]

best = transform.resize(bse_full[sly, slx], ebsd.shape, anti_aliasing=True)
best = np.around(255*best/best.max(), 0).astype(np.uint8)

In [ ]:
plt.close("all")
if x_selected is None:
    x_vals = start_x
else:
    x_vals = start_y
fig = plt.figure(figsize=(17,4))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
ax1.imshow(best, aspect="auto")
ax1.set_title("Output")
ax2.imshow(bse_resized, aspect="auto")
ax2.set_title("Target")
ax3.imshow(bse_resized - best, aspect="auto")
ax3.set_title("Difference")
plt.tight_layout()
# plt.show()